In [0]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline

In [0]:
df1=pd.read_csv("train.csv")
print(df1.shape)
df1.head()

In [0]:
df1.info()

In [0]:
sns.countplot(x='original_author', hue="sentiment_class", data = df1)

In [0]:
sns.countplot( x="sentiment_class", data = df1)

In [0]:
print(df1.lang.nunique())
df1.lang.unique()

In [0]:
print(df1.original_author.nunique())
df1.original_author.unique()

In [0]:
df1.isnull().sum()


In [0]:
y=df1['sentiment_class']
y.shape

***   ***

***retweet_count***

In [0]:
print(df1.retweet_count[0])
isinstance(df1.retweet_count[1], (str))

In [0]:
print(df1.retweet_count.nunique())
df1.retweet_count.unique()

In [0]:
rc=df1['retweet_count']
rc

In [0]:
def convertible(v):
    try:
        int(v)
        return True
    except (TypeError, ValueError):
        return False


res = [int(ele) if convertible(ele)  else int(0) for ele in rc] 
res

In [0]:
res=pd.DataFrame(res)
res.head()

In [0]:
print(res[0].nunique())
res[0].unique()

In [0]:
df1['retweet_count']=res[0]
df1['retweet_count'].unique()

**   **

***Orginal_author***

In [0]:
print(df1.original_author.nunique())
df1.original_author.unique()

In [0]:
oa=df1['original_author']
oa

In [0]:
def convertibles(v):
    try:
        float(v)
        return True
    except (TypeError, ValueError):
        return False


res2 = [str('unkn') if convertibles(ele) else ele  for ele in oa] 
res2

In [0]:
res2=pd.DataFrame(res2)

print(res2[0].nunique())
res2[0].unique()

In [0]:
df1['original_author']=res2[0]
df1['original_author'].nunique()

**   **

**test_data**

In [0]:
df2=pd.read_csv("test.csv")
print(df2.shape)
df2.head()


In [0]:
print(df2.retweet_count.nunique())
df2.retweet_count.unique()

In [0]:
rct=df2['retweet_count']
rct

In [0]:
def convertible(v):
    try:
        int(v)
        return True
    except (TypeError, ValueError):
        return False


rest = [int(ele) if convertible(ele)  else int(0) for ele in rct] 
rest

In [0]:
rest=pd.DataFrame(rest)
rest.head()

In [0]:
print(rest[0].nunique())
rest[0].unique()

In [0]:
df2['retweet_count']=rest[0]
df2['retweet_count'].unique()

**    **

In [0]:
print(df2.original_author.nunique())
#df2.original_author.unique()

In [0]:
oat=df2['original_author']
oat

In [0]:
def convertibles(v):
    try:
        float(v)
        return True
    except (TypeError, ValueError):
        return False


rest2 = [str('unkn') if convertibles(ele) else ele  for ele in oat] 
rest2

In [0]:
rest2=pd.DataFrame(rest2)

print(rest2[0].nunique())
#rest2[0].unique()

In [0]:
df2['original_author']=rest2[0]
df2['original_author'].nunique()

**    **

In [0]:
print(df1.head())
df1.info()

In [0]:
print(df2.head())
df2.info()

In [0]:
np.intersect1d([1,2,3,1,4,3,5,7], [1,3,4,6])        #testing

In [0]:
com=np.intersect1d(df1.original_author, df2.original_author)
com.shape         #246 common in(1528-736)

**   **

***tweets***

In [0]:
x1=df1['original_text']
print(x1.shape)
x1.head

In [0]:
x2=df2['original_text']
print(x2.shape)
x2.tail()

In [0]:
x=pd.concat([x1,x2],axis=0)
x=pd.DataFrame(x)
x.reset_index(inplace=True)
x=x['original_text']
x=pd.DataFrame(x)
print(x.shape)
x.tail()

**Data cleaning**

In [0]:
def remove_pattern1(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    


In [0]:
# remove twitter handles (@user)
x['tidy_tweet'] = np.vectorize(remove_pattern1)(x['original_text'], "@[\w]*")

In [0]:
x.head()

**removing links and mentions**

In [0]:
def remove_pattern(pattern):
  for i in range(x['tidy_tweet'].shape[0]):
    twt=x['tidy_tweet'][i]
    res=twt.split(pattern, maxsplit=1) 
    x['tidy_tweet'][i]=res[0]

In [0]:
remove_pattern('http')

In [0]:
remove_pattern('pic.twitter')

In [0]:
x.head

In [0]:
x['original_text'][7]

In [0]:
x['tidy_tweet'][7]

**   **

**NLTK**

In [0]:
import re
import nltk
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 4622):
    review = re.sub('[^a-zA-Z]', ' ',x['tidy_tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [0]:
corpus[2]

In [0]:
x['tidy_tweet'][2]

**   **

**Embedding Representation**

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras import utils
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout

In [0]:
voc_size=5000

In [0]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [0]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [0]:
embedded_docs[10]

In [0]:
import numpy as np
X_=np.array(embedded_docs)
print(X_.shape)


**   **

In [0]:
f1=df1[['retweet_count','original_author']]
print(f1.shape)
f1.head()


In [0]:
f2=df2[['retweet_count','original_author']]
print(f2.shape)
f2.head()

In [0]:
f=pd.concat([f1,f2],axis=0)
f.reset_index(inplace=True)
f=f.drop("index",axis=1)
print(f.shape)
f.head()

In [0]:
f=pd.get_dummies(f, prefix=['original_author'])
print(f.shape)

In [0]:
X_pd=pd.DataFrame(X_)
X=pd.concat([X_pd,f],axis=1)
X.reset_index(inplace=True)
X=X.drop("index",axis=1)
print(X.shape)
X.tail()

In [0]:
xtrain=X.iloc[:3235,: ].values
xtest=X.iloc[3235:,: ].values
print(xtrain.shape)
xtest.shape

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val  = train_test_split(xtrain,y ,test_size=0.15, random_state=0)

**svm**

In [0]:
from sklearn.svm import SVC
classifier1 = SVC(kernel = 'poly', random_state = 0, degree=4,C=10)
classifier1.fit(X_train, y_train)


In [0]:
from sklearn.model_selection import cross_val_score
acc=cross_val_score(estimator=classifier1, X=X_train, y=y_train , cv=10)

print(acc.mean())
print(acc.std())
acc


In [0]:
y_pred1 = classifier1.predict(X_val)
y_pred1.shape

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred1)
cm

In [0]:
array([[ 21,  89,   7], 
       [ 30, 190,  23],
       [ 10, 101,  15]])

In [0]:
'''from sklearn.model_selection import GridSearchCV
parameters = [{'C':[15,10]}] #svm-2
grid_search = GridSearchCV(estimator = classifier1,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(x, y)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print('done')
print(best_accuracy)
best_parameters'''

In [0]:
226/486

In [0]:
y_pred2 = classifier1.predict(xtest)
y_pred2.shape

In [0]:
ID=df2['id']
ID.shape

In [0]:
pred=pd.DataFrame(y_pred2)
datasets=pd.concat([ID,pred], axis=1)

datasets.columns =['id', 'sentiment_class']
datasets.to_csv('asvm3.csv',index=False)
!cp asvm3.csv "drive/My Drive/"

In [0]:
print(pred[0].value_counts())
sns.countplot( x=-pred[0], data = pred)

**   **

In [0]:
from xgboost import XGBClassifier
classifier4=XGBClassifier(n_estimators=600,reg_lambda=0.1 ,max_depth=4,seed=42,learning_rate=0.1,tree_method='gpu_exact',colsample_bytree=0.9)
classifier4.fit(X_train, y_train)

In [0]:
from sklearn.model_selection import cross_val_score
acc=cross_val_score(estimator=classifier4, X=X_train, y=y_train , cv=10)

print(acc.mean())
print(acc.std())
acc

In [0]:
y_pred1 = classifier4.predict(X_val)
print(y_pred1.shape)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred1)
cm

In [0]:
array([[  9,  97,  11],
       [ 19, 192,  32],
       [ 17,  88,  21]])


In [0]:
y_pred2 = classifier4.predict(xtest)
y_pred2.shape

In [0]:
ID=df2['id']
ID.shape

In [0]:
pred=pd.DataFrame(y_pred2)
datasets=pd.concat([ID,pred], axis=1)

datasets.columns =['id', 'sentiment_class']
datasets.to_csv('axg23.csv',index=False)
!cp axg23.csv "drive/My Drive/"

In [0]:
print(pred[0].value_counts())
sns.countplot( x=pred[0], data = pred)